In [1]:
from dataclasses import dataclass
from dataclasses import field
from enum import Enum
from enum import IntEnum
import itertools
import math
import random
import re
import typing

In [2]:
def res(day, level, result):
    print(f'Result of [Day {day}, Level {level}]: {result}')

In [3]:
with open('../data/08_input_test.txt') as f:
    test_input = f.read()

with open('../data/08_input.txt') as f:
    input = f.read()

In [4]:
@dataclass
class VM:
    acc : int = 0
    pc : int  = 0
    performed_pcs : list = field(default_factory=list)

    # code  ... positional arguments
    # argv  ... vararg argument
    # debug ... keyword-only argument
    def perform(self, code, *argv, debug=False):
        if isinstance(code, str):
            # single OpCode
            if(debug):
                print('Perform OpCode:', code, 'with aguments', *argv if argv else '[no arguments]')
            self.performed_pcs.append(self.pc)
            self._op_codes[code](self, *argv)
            if(debug):
                print('State afterward:', str(self))
        else:
            # line of code
            if(debug):
                print('Perform code:', code)
            while self.pc not in self.performed_pcs:
                if(self.pc + 1 == len(code)):
                    # Reached end of code -> Successful program run
                    return True
                elif(self.pc < 0 or len(code) < self.pc):
                    # Jumped out of bounds of code  -> Unsuccessful program run
                    return False
                # pc is valdi, process the code line
                if(debug):
                    print('Process line:')
                parts = code[self.pc].split()
                op_code = parts[0]
                args = [int(x) for x in parts[1:]]
                self.perform(op_code, *args, debug = debug)
            # Repeating operation -> Unsuccessful program run
            return False

    def _acc_f(self, x):
        self.acc = self.acc + x
        self.pc  = self.pc + 1
    def _jmp_f(self, x):
        self.pc = self.pc + x
    def _nop_f(self, x):
        self.pc = self.pc + 1
    
    _op_codes = {'acc': _acc_f, 'jmp': _jmp_f, 'nop': _nop_f}

In [5]:
# Test singel op code
vm = VM()
print(vm)
vm.perform('acc', 4, debug=True)
vm.perform('nop', 0, debug=True)
vm.perform('nop', 1, debug=True)
vm.perform('jmp' , -3, debug=True)
print(vm)

print("===")
print("===")

# Test code array
vm = VM()
print(vm)
successful = vm.perform(['acc 4', 'nop 0', 'nop 1', 'jmp -3'], debug=True)
print(vm)
print('Successful:', successful)

VM(acc=0, pc=0, performed_pcs=[])
Perform OpCode: acc with aguments 4
State afterward: VM(acc=4, pc=1, performed_pcs=[0])
Perform OpCode: nop with aguments 0
State afterward: VM(acc=4, pc=2, performed_pcs=[0, 1])
Perform OpCode: nop with aguments 1
State afterward: VM(acc=4, pc=3, performed_pcs=[0, 1, 2])
Perform OpCode: jmp with aguments -3
State afterward: VM(acc=4, pc=0, performed_pcs=[0, 1, 2, 3])
VM(acc=4, pc=0, performed_pcs=[0, 1, 2, 3])
===
===
VM(acc=0, pc=0, performed_pcs=[])
Perform code: ['acc 4', 'nop 0', 'nop 1', 'jmp -3']
Process line:
Perform OpCode: acc with aguments 4
State afterward: VM(acc=4, pc=1, performed_pcs=[0])
Process line:
Perform OpCode: nop with aguments 0
State afterward: VM(acc=4, pc=2, performed_pcs=[0, 1])
Process line:
Perform OpCode: nop with aguments 1
State afterward: VM(acc=4, pc=3, performed_pcs=[0, 1, 2])
VM(acc=4, pc=3, performed_pcs=[0, 1, 2])
Successful: True


In [6]:
# Test input
vm = VM()
successfull = vm.perform(test_input.split('\n'), debug=True)
print('Successful:', successfull)
res(8, 'Test', vm.acc)

Perform code: ['nop +0', 'acc +1', 'jmp +4', 'acc +3', 'jmp -3', 'acc -99', 'acc +1', 'jmp -4', 'acc +6']
Process line:
Perform OpCode: nop with aguments 0
State afterward: VM(acc=0, pc=1, performed_pcs=[0])
Process line:
Perform OpCode: acc with aguments 1
State afterward: VM(acc=1, pc=2, performed_pcs=[0, 1])
Process line:
Perform OpCode: jmp with aguments 4
State afterward: VM(acc=1, pc=6, performed_pcs=[0, 1, 2])
Process line:
Perform OpCode: acc with aguments 1
State afterward: VM(acc=2, pc=7, performed_pcs=[0, 1, 2, 6])
Process line:
Perform OpCode: jmp with aguments -4
State afterward: VM(acc=2, pc=3, performed_pcs=[0, 1, 2, 6, 7])
Process line:
Perform OpCode: acc with aguments 3
State afterward: VM(acc=5, pc=4, performed_pcs=[0, 1, 2, 6, 7, 3])
Process line:
Perform OpCode: jmp with aguments -3
State afterward: VM(acc=5, pc=1, performed_pcs=[0, 1, 2, 6, 7, 3, 4])
Successful: False
Result of [Day 8, Level Test]: 5


In [7]:
# Level 1 
vm = VM()
vm.perform(input.split('\n'))

res(8, 1, vm.acc)

Result of [Day 8, Level 1]: 1137


In [8]:
# Level 2
lines = input.split('\n')
successfull = False
for i in range(0, len(lines)):
    transformed_lines = lines.copy()
    if(transformed_lines[i][:3] == 'nop'):
        transformed_lines[i] = transformed_lines[i].replace('nop', 'jmp')
    else:
        transformed_lines[i] = transformed_lines[i].replace('jmp', 'nop')
    
    vm = VM()
    successfull = vm.perform(transformed_lines)
    if successfull:
        res(8, 2, vm.acc)

Result of [Day 8, Level 2]: 1125
